# Filter Quotes

In this notebook we will provide the filtering of the original dataset of quotes, more specifically we are going to filter for the keyword trump and also for quotes with only one QID.

In [5]:
import pandas as pd

Load speaker attributes

In [10]:
speaker_attributes = pd.read_parquet("../data/speaker_attributes_updated.parquet")

speaker_attributes

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Great Britain, United States of America]",[male],1395141751,None,W000178,"[politician, military officer, farmer, cartogr...",[independent politician],None,Q23,George Washington,"[1792 United States presidential election, 178...",item,[Episcopal Church]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[United Kingdom],[male],1395737157,[White British],None,"[playwright, screenwriter, novelist, children'...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Belgium],[male],1380367296,None,None,"[writer, lawyer, librarian, information scient...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[United States of America],[male],1395142029,None,None,"[politician, motivational speaker, autobiograp...",[Republican Party],None,Q207,George W. Bush,"[2000 United States presidential election, 200...",item,"[United Methodist Church, Episcopal Church, Me..."
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Spain],[male],1391704596,None,None,[painter],None,None,Q297,Diego Velázquez,None,item,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055976,[Barker Howard],None,[United States of America],[male],1397399351,None,None,[politician],None,None,Q106406560,Barker B. Howard,None,item,None
9055977,[Charles Macomber],None,[United States of America],[male],1397399471,None,None,[politician],None,None,Q106406571,Charles H. Macomber,None,item,None
9055978,None,[+1848-04-01T00:00:00Z],None,[female],1397399751,None,None,None,None,None,Q106406588,Dina David,None,item,None
9055979,None,[+1899-03-18T00:00:00Z],None,[female],1397399799,None,None,None,None,None,Q106406593,Irma Dexinger,None,item,None


Let's first look at quotes with a certain keyword

In [6]:
def filter_dataframe(word, year):
    """This function will be used to filter the dataset selecting the quotes of a given year containing a given word
    in a free text search fashion. """
    list_df = []
    with pd.read_json("data/quotes-"+year+".json.bz2", lines=True,  chunksize = 10000, compression = 'bz2') as df_reader:
        for chunk in df_reader: #we read chunk by chunk in order not to store everything in memory
            chunk_word = chunk[chunk['quotation'].str.contains(word)] #we create a dataframe containing the word
            list_df.append(chunk_word)  #we append it to the list of dataframes
    df_word = pd.concat(list_df) #we concatenate the dataframes together to obtain a unique one
    return df_word

In [7]:
def removing_not_inherent_quotes(list_of_words, df, aliases):
    """This function removes the quotes containing one of the word in a list of words and 
    whose speaker is one in the aliases list """
    df_copy = df
    for word in list_of_words:
        df_copy = df_copy[~df_copy['quotation'].str.contains(word)] #we remove the quotes containing the word
    df_new = df_copy[~df_copy['speaker'].isin(aliases)] # We remove the quotes whose speaker is in the aliases list
    return df_new

In [ ]:
#we create a list with the years corresponding to the QuoteBank dataset

years_list= ["2015", "2016", "2017", "2018" ,"2019", "2020"]

for year in years_list:
    one_df_Trump = filter_dataframe("Trump", year)
    one_df_Trump.to_csv('df_Trump'+year+'.csv', index=False) #We save the files as checkpoints (it takes a long time to execute)
    print('i finished one file')

Now that we have all of the quotes in separate files, we will join them together

In [8]:
#We create a single dataframe concatenating the previously obtained dataframes

list_df_Trump = []
for year in years_list:
    one_df_Trump = pd.read_csv('df_Trump'+year+'.csv') #we read the file
    list_df_Trump.append(one_df_Trump) #we append the dataframe to the list of dataframes
df_Trump = pd.concat(list_df_Trump) #we concatenate them in a single one

NameError: name 'years_list' is not defined

In [ ]:
df_Trump

Removing the quotes said by Donald Trump himself and the quotes related to the family of Donald Trump or other Trumps

In [ ]:
n_rows_before_cleaning = df_Trump.shape[0] 

In [ ]:
# We look for the aliases of Trump
count = 0
for el in speaker_attributes['aliases']:
    count=count+1
    if type(el) == np.ndarray:
        if ('Trump' in el):
            Trump_aliases = el
            print(Trump_aliases)

In [ ]:
Trump_aliases = Trump_aliases.tolist()
Trump_aliases.append('Donald Trump') # we add Donald Trump to the aliases since it is not present in the list

# we will clean the dataframe removing all the quotes referring to the family of Trump and not to Trump himself
Trump_family = ['Frederick Trump', 'Elizabeth Christ Trump', 'John George Trump',
                                        'Fred Trump', 'Maryanne Trump Barry', 'Mary Trump', 'Ivana Trump', 
                                        'Melania Trump', 'Donald Trump, Jr.', 'Ivanka Trump',
                                        'Eric Trump', 'Tiffany Trump', 'Barron Trump', 'Chrystelle Trump Bond',
                                        'David H. Trump', 'Judd Trump', 'Kelly Trump', 'Peter Trump']
df_Trump = removing_not_inherent_quotes(Trump_family, df_Trump, Trump_aliases)

In [ ]:
df_Trump #we have a look at the dataframe after the cleaning

In [ ]:
n_rows_after_cleaning = df_Trump.shape[0]

In [ ]:
def cleaning_results(n_rows_before, n_rows_after, person):
    # we create a new dataframe for the sake of visualization
    dic = {'before' : [n_rows_before], 'after' : [n_rows_after]}
    df_visual = pd.DataFrame.from_dict(dic)
    #print(df_visual)
    plt.rcParams["figure.figsize"] = (8,6)
    df_visual.mean().plot(kind='bar')
    plt.ylabel('number of quotes')
    plt.title('number of rows before and after cleaning for '+person)
    plt.show()

In [ ]:
cleaning_results(n_rows_before_cleaning, n_rows_after_cleaning, 'Trump')

As we can see from the plot there are a few sentences that we cleaned, the majority of them was already about Donald Trump (not about his family) and not said by Trump himself.

Filter chunk by number of qids, it's not possible to identify authors with quotes that have multiple qids. 

In [11]:
def filter_chunk_by_number_of_qids(initial_chunk, qid_number):
    return initial_chunk[initial_chunk["qids"].str.len() == 1]

In [ ]:
df_Trump = filter_chunk_by_number_of_qids(df_Trump, 1)

df_Trump

Save trump dataset

In [ ]:
# we save it (checkpoint)
df_Trump.to_csv('df_Trump_cleaned.csv', index=False)

### We filter the datset looking for the quotes about Hillary Cllinton

In [ ]:
for year in years_list:
    one_df_Clinton = filter_dataframe("Clinton", year)
    one_df_Clinton.to_pickle('df_Clinton'+year+'.pkl')
    print('i finished one file')

Now we concatenate the dataframe together

In [ ]:
list_df_Clinton = []
for year in years_list:
    one_df_Clinton = pd.read_pickle('df_Clinton'+year+'.pkl')
    list_df_Clinton.append(one_df_Clinton)
df_Clinton = pd.concat(list_df_Clinton)

In [ ]:
df_Clinton.to_csv('df_Clinton.csv', index=False)

In [ ]:
df_Clinton = pd.read_csv('df_Clinton.csv')

In [ ]:
n_rows_before_cleaning = df_Clinton.shape[0]

In [ ]:
count = 0
for el in parquet_frame['aliases']:
    count=count+1
    if type(el) == np.ndarray:
        if ('Clinton' in el):
            Clinton_aliases = el
            print(Clinton_aliases)

In [ ]:
df_Clinton = removing_not_inherent_quotes(['William Jefferson Clinton', 'William J. Clinton', 'Bill Clinton',
                                        'William Clinton', 'President Clinton', 'President Bill Clinton', 
                                        'George William Clinton'], 
                                        df_Clinton, ['Clinton', 'Hillary Clinton'])

In [ ]:
n_rows_after_cleaning = df_Clinton.shape[0]

In [ ]:
cleaning_results(n_rows_before_cleaning, n_rows_after_cleaning, 'Clinton')

In [ ]:
df_Clinton.to_csv('df_Clinton_cleaned.csv', index=False)

In [ ]:
df_Clinton #we have a look at the dataframe